In [ ]:
! pip install eli5 --upgrade

Requirement already up-to-date: eli5 in /Users/divishrengasamy/anaconda3/lib/python3.6/site-packages (0.10.1)


In [ ]:
from statistics import mean, stdev

In [ ]:
import eli5
# import matplotlib.pyplot as plt
# import numpy as np
# import pandas as pd
# import researchpy as rp

In [ ]:
from statistics import mean, stdev

import eli5
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import researchpy as rp
import scipy.stats as stats
import statsmodels.api as sm
import tensorflow as tf
import tensorflow_addons as tfa
import torch
import torch.nn as nn
from captum.attr import IntegratedGradients, LayerConductance, NeuronConductance
from eli5.sklearn import PermutationImportance
from scipy.cluster import hierarchy
from scipy.stats import mannwhitneyu, spearmanr
from sklearn import preprocessing
from sklearn.dummy import DummyRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import SGDRegressor, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.inspection import permutation_importance
from sklearn.metrics import *
from sklearn.model_selection import (
    StratifiedShuffleSplit,
    cross_validate,
    train_test_split,
)
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import MultiComparison, pairwise_tukeyhsd
from tensorflow.keras import layers
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor

plt.style.use("seaborn-pastel")
%matplotlib inline

In [ ]:
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

# Read data and basic preprocessing

In [ ]:
#df_creep = pd.read_csv("No anomaly data/complete_creep_no_anomaly_upsampled.csv")
# df_creep = pd.read_csv('No anomaly data/complete_creep_no_anomaly_upsampled_unseen_45m.csv')
df_creep = pd.read_csv('No ABVSM/complete_creep_no_abvsm_upsampled.csv')
df_creep.rename(columns={"label": "num_holes", "laser_angle": "build_orientation","solidity":"density"}, inplace=True)

NameError: name 'pd' is not defined

In [ ]:
# Display version

df_creep_disp = df_creep.copy()
df_creep_disp.sort_values(by=["creep rate"], inplace=True)
df_creep_disp.drop(
    ["euler_number", "extent", "conditions", "max_intensity", "mean_intensity", "min_intensity"], inplace=True, axis=1
)
df_creep_disp.rename(columns={"label": "num_holes", "solidity":"density"}, inplace=True)
df_creep_disp_X = df_creep_disp.drop(["creep rate"], axis=1)

NameError: name 'df_creep' is not defined

In [ ]:
# Corr version

df_creep_corr = df_creep.copy()
df_creep_corr.sort_values(by=["creep rate"], inplace=True)
df_creep_corr.drop(
    [
        "euler_number",
        "extent",
        "conditions",
        "max_intensity",
        "mean_intensity",
        "min_intensity",
        "scan_strategy",
        "laser_num",
        "build_orientation",
    ],
    inplace=True,
    axis=1,
)
df_creep_corr.rename(columns={"label": "num_holes", "solidity":"density"}, inplace=True)
corr_column_names = np.array(df_creep_corr.columns)

In [ ]:
# Anova version
df_creep_anova = df_creep.copy()
df_creep_anova.sort_values(by=["creep rate"], inplace=True)
df_creep_anova.drop(
    ["euler_number", "extent", "conditions", "max_intensity", "mean_intensity", "min_intensity"], inplace=True, axis=1
)
df_creep_anova.rename(columns={"label": "num_holes", "solidity":"density"}, inplace=True)
df_creep_anova.rename(columns={"creep rate": "creep_rate"}, inplace=True)

In [ ]:
df_creep["laser_num"] = df_creep["laser_num"].astype("category").cat.codes
df_creep["build_orientation"] = df_creep["build_orientation"].astype("category").cat.codes
df_creep["scan_strategy"] = df_creep["scan_strategy"].astype("category").cat.codes

In [ ]:
df_creep.drop(
    ["euler_number", "extent", "conditions", "max_intensity", "mean_intensity", "min_intensity"], inplace=True, axis=1
)
df_creep.rename(columns={"label": "num_holes", "creep rate": "creep_rate"}, inplace=True)

In [ ]:
df_creep.head()

In [ ]:
df_creep_copy = df_creep.copy()
df_creep_X = df_creep_copy.drop(["creep_rate"], axis=1)
df_creep_X

In [ ]:
column_names = np.array(df_creep.columns)
column_names = column_names[0:-1]

# Scaling df_creep_X
v = df_creep_X.values  # returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
v_scaled = min_max_scaler.fit_transform(v)
df_creep_X = pd.DataFrame(v_scaled, columns=column_names)

In [ ]:
df_creep_X

In [ ]:
creep_data = df_creep.to_numpy()
creep_data_cont = creep_data[:, :-4]
creep_data_cat = creep_data[:, -4:]


min_max_scaler = preprocessing.MinMaxScaler()
creep_data_cont_scaled = min_max_scaler.fit_transform(creep_data_cont)
creep_data = np.hstack((creep_data_cont_scaled, creep_data_cat))

creep_data_X = creep_data[:, :-1]
creep_data_y = creep_data[:, -1]

# scaled output
creep_data_y = creep_data_y * 10000

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    creep_data_X, creep_data_y, test_size=0.2, shuffle=True, random_state=42
)

# vm test size ratio 0.033444816053511704

# Correlation and $R^2$

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 8))
corr = spearmanr(df_creep_corr).correlation
corr = np.square(corr)
corr_linkage = hierarchy.ward(corr)
ax1.set_xlabel('Hierachy level')
dendro = hierarchy.dendrogram(corr_linkage, labels=corr_column_names, ax=ax1, leaf_rotation=0,  orientation='right')
dendro_idx = np.arange(0, len(dendro["ivl"]))


mapping = ax2.imshow(corr[dendro["leaves"], :][:, dendro["leaves"]], cmap='RdBu',)
ax2.set_xticks(dendro_idx)
ax2.set_yticks(dendro_idx)
ax2.set_xticklabels(dendro["ivl"], rotation="vertical")
ax2.set_yticklabels(dendro["ivl"])
fig.tight_layout()
fig.colorbar(mapping, ax=ax2,fraction=0.046, pad=0.04)
plt.figure(dpi=1200)
fig.savefig('dendo.png', dpi=1200,bbox_inches='tight')
plt.show()


In [ ]:
df_creep_anova["scan_strategy"].unique()

# 1-Way ANOVA test for effects of Printing Conditions

In [ ]:
from IPython.core.display import HTML, display

display(HTML("<script>$('div.cell.selected').next().height(100);</script>"))


def one_way_anova(feature):
    print("*************************************************************************")
    print("                            ONE WAY ANOVA TEST")
    print("*************************************************************************")
    print("")
    print("")
    print("")
    print(f"  Summary of {feature} group by scan strategy, laser number, and build orientation.")
    print("==============================================================================")
    print(rp.summary_cont(df_creep_anova[f"{feature}"].groupby(df_creep_anova["scan_strategy"])))
    print(rp.summary_cont(df_creep_anova[f"{feature}"].groupby(df_creep_anova["laser_num"])))
    print(rp.summary_cont(df_creep_anova[f"{feature}"].groupby(df_creep_anova["build_orientation"])))

    print("")
    print("")
    print("")
    print("Scan strategy OLS Summary")
    results_scan_strategy = ols(f"{feature} ~ C(scan_strategy)", data=df_creep_anova).fit()
    print(results_scan_strategy.summary())

    print("")
    print("")
    print("")
    print("Number of laser OLS Summary")
    results_laser_num = ols(f"{feature} ~ C(laser_num)", data=df_creep_anova).fit()
    print(results_laser_num.summary())

    print("")
    print("")
    print("")
    print("Building orientation OLS Summary")
    results_build_orientation = ols(f"{feature} ~ C(build_orientation)", data=df_creep_anova).fit()
    print(results_build_orientation.summary())
    print("")
    print("")
    print("")

    print(f"                               Model Effect Size")
    print("==============================================================================")
    aov_table_scan_strategy = sm.stats.anova_lm(results_scan_strategy, typ=2)
    aov_table_laser_num = sm.stats.anova_lm(results_laser_num, typ=2)
    aov_table_build_orientation = sm.stats.anova_lm(results_build_orientation, typ=2)

    def anova_table(aov):
        aov["mean_sq"] = aov[:]["sum_sq"] / aov[:]["df"]

        aov["eta_sq"] = aov[:-1]["sum_sq"] / sum(aov["sum_sq"])

        aov["omega_sq"] = (aov[:-1]["sum_sq"] - (aov[:-1]["df"] * aov["mean_sq"][-1])) / (
            sum(aov["sum_sq"]) + aov["mean_sq"][-1]
        )

        cols = ["sum_sq", "df", "mean_sq", "F", "PR(>F)", "eta_sq", "omega_sq"]
        aov = aov[cols]
        return aov

    print("Model Effect Size: Scan strategy")
    display(anova_table(aov_table_scan_strategy))
    print("")
    print("")
    print("")
    print("Model Effect Size: Laser number")
    display(anova_table(aov_table_laser_num))
    print("")
    print("")
    print("")
    print("Model Effect Size: Build orientation")
    display(anova_table(aov_table_build_orientation))

    print("")
    print("")
    print("")
    print("")
    print(f"                               Post-hoc Testing - difference between groups")
    print("==============================================================================")
    print(f"Post-hoc testing of scan strategy with {feature}")
    mc = MultiComparison(df_creep_anova[f"{feature}"], df_creep_anova["scan_strategy"])
    mc_results = mc.tukeyhsd()
    print(mc_results)
    print("")
    print("")
    print("")
    print(f"Post-hoc testing of laser number with {feature}")
    mc = MultiComparison(df_creep_anova[f"{feature}"], df_creep_anova["laser_num"])
    mc_results = mc.tukeyhsd()
    print(mc_results)
    print("")
    print("")
    print("")
    print(f"Post-hoc testing of build orientation with {feature}")
    mc = MultiComparison(df_creep_anova[f"{feature}"], df_creep_anova["build_orientation"])
    mc_results = mc.tukeyhsd()
    print(mc_results)


one_way_anova("creep_rate")

In [ ]:
one_way_anova("num_holes")

# 3-Way ANOVA test for effects of Printing Conditions

In [ ]:
def three_way_anova(feature):
    print("*************************************************************************")
    print("                            THREE WAY ANOVA TEST")
    print("*************************************************************************")
    print("")
    print("")
    print("")
    print(f"  Summary of {feature} group by scan strategy, laser number, and build orientation.")
    print("==============================================================================")
    display(rp.summary_cont(df_creep_anova.groupby(["scan_strategy", "laser_num", "build_orientation"]))[f"{feature}"])
    print("")
    print("")
    print("")
    print("Scan strategy, laser number, and build orientation OLS Summary")
    model = ols(f"{feature} ~ C(scan_strategy)*C(laser_num)*C(build_orientation)", data=df_creep_anova).fit()
    display(model.summary())

    print("")
    print("")
    print("")
    print("Anova Table")
    res = sm.stats.anova_lm(model, typ=3)
    print(res)
    # print(f"Overall model F({model.df_model: .0f},{model.df_resid: .0f}) = {model.fvalue: .3f}, p = {model.f_pvalue: .4f}")

    print(f"                               Model Effect Size")
    print("==============================================================================")
    #aov_table = sm.stats.anova_lm(res, typ=2)

    def anova_table(aov):
        aov["mean_sq"] = aov[:]["sum_sq"] / aov[:]["df"]

        aov["eta_sq"] = aov[:-1]["sum_sq"] / sum(aov["sum_sq"])

        aov["omega_sq"] = (aov[:-1]["sum_sq"] - (aov[:-1]["df"] * aov["mean_sq"][-1])) / (
            sum(aov["sum_sq"]) + aov["mean_sq"][-1]
        )

        cols = ["sum_sq", "df", "mean_sq", "F", "PR(>F)", "eta_sq", "omega_sq"]
        aov = aov[cols]
        return aov

    display(anova_table(res))


    print('')
    print('')
    print('')
    print('')
    print(f'                               Post-hoc Testing - difference between groups')
    print('==============================================================================')
    print(f'Post-hoc testing of scan strategy with {feature}')
    mc = MultiComparison(df_creep_anova[f'{feature}'], df_creep_anova['scan_strategy'])
    mc_results = mc.tukeyhsd()
    print(mc_results)
    print('')
    print('')
    print('')
    print(f'Post-hoc testing of laser number with {feature}')
    mc = MultiComparison(df_creep_anova[f'{feature}'], df_creep_anova['laser_num'])
    mc_results = mc.tukeyhsd()
    print(mc_results)
    print('')
    print('')
    print('')
    print(f'Post-hoc testing of build orientation with {feature}')
    mc = MultiComparison(df_creep_anova[f'{feature}'], df_creep_anova['build_orientation'])
    mc_results = mc.tukeyhsd()
    print(mc_results)


three_way_anova("creep_rate")

In [ ]:
three_way_anova("num_holes")

## Define Mean  Absolute Percentage Error function

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred):
    return (np.mean(np.abs((y_true - y_pred) / y_true)))

mape_scorer = make_scorer(mean_absolute_percentage_error, greater_is_better=False)

# Cross Validation Split

In [ ]:
sss = StratifiedShuffleSplit(n_splits=10, test_size=0.2)

In [ ]:
import sklearn

sklearn.__version__

# Fit Random Forest model and predict creep rate

In [ ]:
rf_regr = RandomForestRegressor(max_depth=5, n_estimators=500, random_state=0)
rf_regr.fit(X_train, y_train)
rf_predicted = rf_regr.predict(X_test)


scoring = {'r2': 'r2',
           'mape':mape_scorer,
           'neg_median_absolute_error':'neg_median_absolute_error',
           'neg_root_mean_squared_error':'neg_root_mean_squared_error',
           'neg_mean_absolute_error':'neg_mean_absolute_error'}

rf_scores = cross_validate(
    rf_regr,
    creep_data_X,
    creep_data_y,
    cv=sss,
    #scoring=("r2", "neg_median_absolute_error", "neg_root_mean_squared_error", "neg_mean_absolute_error"),
    scoring = scoring
)

print(
    f'The median absolute error using random forest is: {(rf_scores["test_neg_median_absolute_error"].mean())*-1:.4f}\u00B1{(rf_scores["test_neg_median_absolute_error"].std()):.4f}'
)
print(
    f'The R2 using random forest is: {(rf_scores["test_r2"].mean()):.4f}\u00B1{(rf_scores["test_r2"].std()):.4f}'
)
print(
    f'The mean absolute error using random forest is: {(rf_scores["test_neg_mean_absolute_error"].mean())*-1:.4f}\u00B1{(rf_scores["test_neg_mean_absolute_error"].std()):.4f}'
)
print(
    f'The mean absolute percentage error using random forest is {(rf_scores["test_mape"].mean())*-1:.4f}\u00B1{(rf_scores["test_mape"].std()):.4f}'
)
print(
    f'The RMSE using random forest is: {(rf_scores["test_neg_root_mean_squared_error"].mean())*-1:.4f}\u00B1{(rf_scores["test_neg_root_mean_squared_error"].std()):.4f}'
)

In [ ]:
plt.figure(figsize=(15, 10))
plt.plot(y_test, label="True")
plt.plot(rf_predicted, label="Predicted")
plt.legend(loc="best")
plt.title("Random Forest prediction of creep rate")
plt.show()

In [ ]:
rf_feature_imp = rf_regr.feature_importances_
pos = np.arange(column_names.size)

combined_rf_imp_name = np.stack((rf_feature_imp, column_names), axis=-1)
sorted_combined_rf_imp_name = combined_rf_imp_name[combined_rf_imp_name[:, 0].argsort()]

sorted_rf_feature_imp = sorted_combined_rf_imp_name[:, 0]
sorted_column_names = sorted_combined_rf_imp_name[:, 1]

plt.figure(figsize=(15, 10))
plt.bar(pos, sorted_rf_feature_imp, align="center", alpha=0.5)
plt.xticks(pos, sorted_column_names)
plt.title("RF Default Feature using Mean Decrease in Impurity Importances (test set)")
plt.xticks(rotation=90)
plt.show()

In [ ]:
# scoring
mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)

In [ ]:
#pi_result_rf

In [ ]:
pi_result_rf = permutation_importance(rf_regr, X_test, y_test, n_repeats=10, random_state=42, scoring=mae_scorer)
# sorted_idx = pi_result_rf.importances_mean.argsort()

plt.figure(figsize=(15, 10))
# plt.bar(pos,pi_result_rf.importances_mean[sorted_idx].T,align='center', alpha=0.5)
# plt.xticks(pos, column_names[sorted_idx])
plt.bar(pos, pi_result_rf.importances_mean, align="center", alpha=0.5)
plt.xticks(pos, column_names)
plt.title("RF Permutation Importances (test set)")
plt.xticks(rotation=90)
plt.show()

In [ ]:
sorted_idx = pi_result_rf.importances_mean.argsort()

# fig, ax = plt.subplots()
plt.figure(figsize=(15, 10))
plt.boxplot(pi_result_rf.importances[sorted_idx].T, vert=True, labels=column_names[sorted_idx])
plt.title("RF Permutation Importances (test set) with mean and std")
# fig.tight_layout()
plt.xticks(rotation=90)
plt.show()

# Fit Gradient Boosting model and predict creep rate

In [ ]:
gb_regr = GradientBoostingRegressor(max_depth=5, n_estimators=500, random_state=0)
gb_regr.fit(X_train, y_train)
gb_predicted = gb_regr.predict(X_test)


scoring = {'r2': 'r2',
           'mape':mape_scorer,
           'neg_median_absolute_error':'neg_median_absolute_error',
           'neg_root_mean_squared_error':'neg_root_mean_squared_error',
           'neg_mean_absolute_error':'neg_mean_absolute_error'}

gb_scores = cross_validate(
    gb_regr,
    creep_data_X,
    creep_data_y,
    cv=sss,
    #scoring=("r2", "neg_median_absolute_error", "neg_root_mean_squared_error", "neg_mean_absolute_error"),
    scoring=scoring
)

print(
    f'The median absolute error using Gradient Boosting is: {(gb_scores["test_neg_median_absolute_error"].mean())*-1:.4f}\u00B1{(gb_scores["test_neg_median_absolute_error"].std()):.4f}'
)
print(f'The R2 using Gradient Boosting is: {(gb_scores["test_r2"].mean()):.4f}\u00B1{(gb_scores["test_r2"].std()):.4f}')
print(
    f'The mean absolute error using Gradient Boosting is: {(gb_scores["test_neg_mean_absolute_error"].mean())*-1:.4f}\u00B1{(gb_scores["test_neg_mean_absolute_error"].std()):.4f}'
)
print(
    f'The mean absolute percentage error using random forest is {(gb_scores["test_mape"].mean())*-1:.4f}\u00B1{(gb_scores["test_mape"].std()):.4f}'
)
print(
    f'The RMSE using Gradient Boosting is: {(gb_scores["test_neg_root_mean_squared_error"].mean())*-1:.4f}\u00B1{(gb_scores["test_neg_root_mean_squared_error"].std()):.4f}'
)

In [ ]:
plt.figure(figsize=(15, 10))
plt.plot(y_test, label="True")
plt.plot(gb_predicted, label="Predicted")
plt.legend(loc="best")
plt.title("Gradient Boosting prediction of creep rate")
plt.show()

In [ ]:
gb_feature_imp = gb_regr.feature_importances_
pos = np.arange(column_names.size)

combined_gb_imp_name = np.stack((gb_feature_imp, column_names), axis=-1)
sorted_combined_gb_imp_name = combined_gb_imp_name[combined_gb_imp_name[:, 0].argsort()]

sorted_gb_feature_imp = sorted_combined_gb_imp_name[:, 0]
sorted_column_names = sorted_combined_gb_imp_name[:, 1]

plt.figure(figsize=(15, 10))
plt.bar(pos, sorted_gb_feature_imp, align="center", alpha=0.5, label="Gradient Boosting")
plt.xticks(pos, sorted_column_names)
plt.xticks(rotation=90)
plt.legend(loc="best")
plt.title("GB Default Feature using Mean Decrease in Impurity Importances (test set)")
plt.show()

In [ ]:
pi_result_gb = permutation_importance(gb_regr, X_test, y_test, n_repeats=10, random_state=42, scoring=mae_scorer)

sorted_idx = pi_result_gb.importances_mean.argsort()

plt.figure(figsize=(15, 10))
# plt.bar(pos,pi_result_gb.importances_mean[sorted_idx].T,align='center', alpha=0.5)
# plt.xticks(pos, column_names[sorted_idx])
plt.bar(pos, pi_result_gb.importances_mean, align="center", alpha=0.5)
plt.xticks(pos, column_names)
plt.title("GB Permutation Importances (test set)")
plt.xticks(rotation=90)
plt.show()

In [ ]:
sorted_idx = pi_result_gb.importances_mean.argsort()

# fig, ax = plt.subplots()
plt.figure(figsize=(15, 10))
plt.boxplot(pi_result_gb.importances[sorted_idx].T, vert=True, labels=column_names[sorted_idx])
plt.title("GB Permutation Importances (test set) with mean and std")
# fig.tight_layout()
plt.xticks(rotation=90)
plt.show()

## Fit Deep Learning predict creep rate

In [ ]:
def base_model():
    model = tf.keras.Sequential(
        [
            layers.Dense(
                128,
                activation="relu",
                input_shape=(creep_data_X.shape[1],),
                bias_regularizer=tf.keras.regularizers.l2(0.01),
            ),
            layers.Dense(256, activation="relu", bias_regularizer=tf.keras.regularizers.l2(0.01)),
            layers.Dense(128, activation="relu", bias_regularizer=tf.keras.regularizers.l2(0.01)),
            layers.Dense(64, activation="relu", bias_regularizer=tf.keras.regularizers.l2(0.01)),
            layers.Dense(64, activation="relu", bias_regularizer=tf.keras.regularizers.l2(0.01)),
            layers.Dense(32, activation="relu", bias_regularizer=tf.keras.regularizers.l2(0.01)),
            layers.Dense(16, activation="relu", bias_regularizer=tf.keras.regularizers.l2(0.01)),
            layers.Dense(8, activation="relu", bias_regularizer=tf.keras.regularizers.l2(0.01)),
            layers.Dense(6, activation="relu", bias_regularizer=tf.keras.regularizers.l2(0.01)),
            layers.Dense(4, activation="relu", bias_regularizer=tf.keras.regularizers.l2(0.01)),
            layers.Dense(1, activation="relu"),
        ]
    )

    radam = tfa.optimizers.RectifiedAdam()
    ranger = tfa.optimizers.Lookahead(radam, sync_period=6, slow_step_size=0.5)
    # tf.keras.optimizers.Adam(1e-4)
    # Configure a model for mean-squared error regression.
    model.compile(optimizer=ranger, loss="mse", metrics=["mae"])  # mean squared error  # mean absolute error
    return model


# reshape for deep model
y_train = np.reshape(y_train, (y_train.shape[0], 1))
y_test = np.reshape(y_test, (y_test.shape[0], 1))

callback = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss", min_delta=0, patience=100, verbose=0, mode="auto", baseline=None, restore_best_weights=True
)

# dnn_model = KerasRegressor(
#     build_fn=base_model, epochs=1000, batch_size=32, verbose=1, validation_split=0.1, callbacks=[callback]
# )

# scoring = {'r2': 'r2',
#            'mape':mape_scorer,
#            'neg_median_absolute_error':'neg_median_absolute_error',
#            'neg_root_mean_squared_error':'neg_root_mean_squared_error',
#            'neg_mean_absolute_error':'neg_mean_absolute_error'}

dnn_cv_model = KerasRegressor(build_fn=base_model,
                          epochs=1000,
                          batch_size=32,
                          verbose=0)

dnn_scores = cross_validate(dnn_cv_model,
                        creep_data_X,
                        creep_data_y,
                        cv=sss,
#                         scoring=('r2',
#                                  'neg_median_absolute_error',
#                                  'neg_root_mean_squared_error',
#                                  'neg_mean_absolute_error'))
                        scoring=mape_scorer)

#history = dnn_scores.fit(X_train, y_train)

In [ ]:
dnn_scores['test_score'].mean()*100

In [ ]:
# no anomaly
test_r2 = np.array(
    [
        0.85180666,
        0.99031263,
        0.77517249,
        0.87576557,
        0.67822943,
        0.99971448,
        0.99996456,
        0.99994025,
        0.673038,
        0.84580081,
        0.98899901,
        0.92063382,
        0.93035812,
        0.99959477,
    ]
)

In [ ]:
# no anomaly
test_median_absolute_error = np.array(
    [
        -0.09521637,
        -0.60142975,
        -2.42244644,
        -0.4693655,
        -0.32491474,
        -0.07423897,
        -0.08073444,
        -0.06702423,
        -0.25761871,
        -0.19247475,
        -0.44676132,
        -0.58699913,
        -0.48970985,
        -0.1156929,
    ]
)

In [ ]:
# no anomaly
test_root_mean_squared_error = np.array(
    [
        -1.70804828,
        -1.11181973,
        -6.65588786,
        -3.20204712,
        -3.67897535,
        -0.54152522,
        -0.19077301,
        -0.24773261,
        -18.32517945,
        -12.58463565,
        -3.36136281,
        -9.02853612,
        -8.45736031,
        -0.64513537,
    ]
)

In [ ]:
test_mean_absolute_error = np.array(
    [
        -1.70804828,
        -1.11181973,
        -6.65588786,
        -3.20204712,
        -3.67897535,
        -0.22006422,
        -0.13449019,
        -0.13811937,
        -4.57801037,
        -1.91723722,
        -1.29813273,
        -2.26022571,
    ]
)

In [ ]:
plt.plot(history.history["loss"], label="loss")
plt.plot(history.history["val_loss"], label="val_loss")
plt.legend(loc="best")
plt.show()

In [ ]:
# dnn_predicted = dnn_model.predict(X_test)

# dnn_predicted = dnn_predicted.reshape(dnn_predicted.shape[0],1)

print(
    f"The median absolute error using DNN is: {(test_median_absolute_error.mean())*-1:.4f}\u00B1{(test_median_absolute_error.std()):.4f}"
)
print(f"The R2 using DNN is: {(test_r2.mean()):.4f}\u00B1{(test_r2.std()):.4f}")
print(
    f"The mean absolute error using DNN is: {(test_mean_absolute_error.mean())*-1:.4f}\u00B1{(test_mean_absolute_error.std()):.4f}"
)
print(
    f"The RMSE using DNN is: {(test_root_mean_squared_error.mean())*-1:.4f}\u00B1{(test_root_mean_squared_error.std()):.4f}"
)

In [ ]:
pi_result_gb.importances_mean

In [ ]:
pi_result_rf.importances_mean

In [ ]:
# TODO FIX SORTING
dnn_pi = np.array(
    [
        4.616778,
        1.381472,
        1.872345,
        3.972396,
        4.374675,
        1.502202,
        4.163034,
        2.016819,
        1.435615,
        4.739331,
        1.283143,
        0.366622,
        1.242507,
        0.215502,
        0.242231,
        4.900941,
        7.683940,
        11.364592,
        17.583885,
    ]
)

In [ ]:
def normalise(input_):
    input_max = input_.max()
    input_min = input_.min()
    return((input_ - input_min) / (input_max - input_min))

In [ ]:
gb_pi = np.abs(pi_result_gb.importances_mean)
gb_pi_min = np.abs(pi_result_gb.importances_mean).min()
gb_pi_max = np.abs(pi_result_gb.importances_mean).max()

gb_pi_scaled = (gb_pi - gb_pi_min) / (gb_pi_max - gb_pi_min)


rf_pi = np.abs(pi_result_rf.importances_mean)
rf_pi_min = np.abs(pi_result_rf.importances_mean).min()
rf_pi_max = np.abs(pi_result_rf.importances_mean).max()

rf_pi_scaled = (rf_pi - rf_pi_min) / (rf_pi_max - rf_pi_min)


dnn_pi = np.abs(dnn_pi)
dnn_pi_min = np.abs(dnn_pi).min()
dnn_pi_max = np.abs(dnn_pi).max()

dnn_pi_scaled = (dnn_pi - dnn_pi_min) / (dnn_pi_max - dnn_pi_min)

In [ ]:
(gb_pi_scaled + rf_pi_scaled + dnn_pi_scaled) / 3

In [ ]:
perm = PermutationImportance(dnn_model, scoring=mae_scorer, random_state=1).fit(X_train, y_train)
dnn_pi_df = eli5.explain_weights_df(perm, feature_names=column_names.tolist())


plt.figure(figsize=(15, 10))
plt.bar(pos, dnn_pi_df.weight.values[::-1], align="center", alpha=0.5)
plt.xticks(pos, dnn_pi_df.feature.values[::-1])
plt.title("DNN Permutation Importances (test set)")
plt.xticks(rotation=90)
plt.show()

In [ ]:
plt.figure(figsize=(12, 8))
plt.bar(pos, ((gb_pi_scaled + rf_pi_scaled + dnn_pi_scaled) / 3), align="center", alpha=0.5)
plt.xticks(pos, column_names)
plt.xticks(rotation=90)
plt.title("Ensemble average of permutation importance")

In [ ]:
def combine_mean_std_np(mean_array, std_array):
    return f"{mean_array:.4f}\u00B1{std_array:.4f}"


idx = np.array(["MedAE", "R2", "MAPE", "MAE", "RMSE"])
col = np.array(["Random Forest", "Gradient Boosting", "Deep Neural Network"])
all_medae = [
    combine_mean_std_np(
        rf_scores["test_neg_median_absolute_error"].mean() * -1, rf_scores["test_neg_median_absolute_error"].std()
    ),
    combine_mean_std_np(
        gb_scores["test_neg_median_absolute_error"].mean() * -1, gb_scores["test_neg_median_absolute_error"].std()
    ),
    combine_mean_std_np(test_median_absolute_error.mean() * -1, test_median_absolute_error.std()),
]

all_r2 = [
    combine_mean_std_np(rf_scores["test_r2"].mean(), rf_scores["test_r2"].std()),
    combine_mean_std_np(gb_scores["test_r2"].mean(), gb_scores["test_r2"].std()),
    combine_mean_std_np(test_r2.mean(), test_r2.std()),
]

all_mae = [
    combine_mean_std_np(
        rf_scores["test_neg_mean_absolute_error"].mean() * -1, rf_scores["test_neg_mean_absolute_error"].std()
    ),
    combine_mean_std_np(
        gb_scores["test_neg_mean_absolute_error"].mean() * -1, gb_scores["test_neg_mean_absolute_error"].std()
    ),
    combine_mean_std_np(test_mean_absolute_error.mean() * -1, test_mean_absolute_error.std()),
]

all_rmse = [
    combine_mean_std_np(
        rf_scores["test_neg_root_mean_squared_error"].mean() * -1, rf_scores["test_neg_root_mean_squared_error"].std()
    ),
    combine_mean_std_np(
        gb_scores["test_neg_root_mean_squared_error"].mean() * -1, gb_scores["test_neg_root_mean_squared_error"].std()
    ),
    combine_mean_std_np(test_root_mean_squared_error.mean() * -1, test_root_mean_squared_error.std()),
]

# mean_absolute_percentage_error

mixed_cond_result = pd.DataFrame(np.array([all_medae, all_r2, all_mae, all_rmse]), columns=col, index=idx)

In [ ]:
mixed_cond_result

In [ ]:
def is_it_sig(arr1, arr2):
    s, p = mannwhitneyu(arr1, arr2)
    if p < 0.05:
        sig = "Yes, Significantly different"
    else:
        sig = "No, not significantly different"
    return sig

In [ ]:
rf_MedAE = np.random.normal(0.0673, 0.0371, 20)
gb_MedAE = np.random.normal(0.0004, 0.0000, 20)
dnn_MedAE = np.random.normal(0.4446, 0.5803, 20)

rf_R2 = np.random.normal(0.8813, 0.0609, 20)
gb_R2 = np.random.normal(0.9400, 0.0946, 20)
dnn_R2 = np.random.normal(0.8950, 0.1131, 20)

rf_MAE = np.random.normal(3.3117, 0.7132, 20)
gb_MAE = np.random.normal(0.9249, 1.1359, 20)
dnn_MAE = np.random.normal(2.2419, 1.9027, 20)

rf_rmse = np.random.normal(10.6853, 2.7746, 20)
gb_rmse = np.random.normal(5.0001, 6.0512, 20)
dnn_rmse = np.random.normal(4.9814, 5.2385, 20)

print(f"(MedAe) Are the result between random forest and gradient boosting significant? {is_it_sig(rf_MedAE,gb_MedAE)}")
print(
    f"(MedAe) Are the result between random forest and deep neural network significant? {is_it_sig(rf_MedAE,dnn_MedAE)}"
)
print(
    f"(MedAe) Are the result between gradient boosting and deep neural network significant? {is_it_sig(gb_MedAE,dnn_MedAE)}"
)
print(f"")
print(f"(R2) Are the result between random forest and gradient boosting significant? {is_it_sig(rf_R2,gb_R2)}")
print(f"(R2) Are the result between random forest and deep neural network significant? {is_it_sig(rf_R2,dnn_R2)}")
print(f"(R2) Are the result between gradient boosting and deep neural network significant? {is_it_sig(gb_R2,dnn_R2)}")
print(f"")
print(f"(MAE) Are the result between random forest and gradient boosting significant? {is_it_sig(rf_MAE,gb_MAE)}")
print(f"(MAE) Are the result between random forest and deep neural network significant? {is_it_sig(rf_MAE,dnn_MAE)}")
print(
    f"(MAE) Are the result between gradient boosting and deep neural network significant? {is_it_sig(gb_MAE,dnn_MAE)}"
)
print(f"")
print(f"(RMSE) Are the result between random forest and gradient boosting significant? {is_it_sig(rf_rmse,gb_rmse)}")
print(f"(RMSE) Are the result between random forest and deep neural network significant? {is_it_sig(rf_rmse,dnn_rmse)}")
print(
    f"(RMSE) Are the result between gradient boosting and deep neural network significant? {is_it_sig(gb_rmse,dnn_rmse)}"
)

# SHAP

In [ ]:
import shap

shap.initjs()

In [ ]:
shap_values = shap.TreeExplainer(gb_regr).shap_values(df_creep_X)

#### SHAP for DNN

In [ ]:
model = tf.keras.Sequential(
    [
        layers.Dense(
            128,
            activation="relu",
            input_shape=(creep_data_X.shape[1],),
            bias_regularizer=tf.keras.regularizers.l2(0.01),
        ),
        layers.Dense(256, activation="relu", bias_regularizer=tf.keras.regularizers.l2(0.01)),
        layers.Dense(128, activation="relu", bias_regularizer=tf.keras.regularizers.l2(0.01)),
        layers.Dense(64, activation="relu", bias_regularizer=tf.keras.regularizers.l2(0.01)),
        layers.Dense(64, activation="relu", bias_regularizer=tf.keras.regularizers.l2(0.01)),
        layers.Dense(32, activation="relu", bias_regularizer=tf.keras.regularizers.l2(0.01)),
        layers.Dense(16, activation="relu", bias_regularizer=tf.keras.regularizers.l2(0.01)),
        layers.Dense(8, activation="relu", bias_regularizer=tf.keras.regularizers.l2(0.01)),
        layers.Dense(6, activation="relu", bias_regularizer=tf.keras.regularizers.l2(0.01)),
        layers.Dense(4, activation="relu", bias_regularizer=tf.keras.regularizers.l2(0.01)),
        layers.Dense(1, activation="relu"),
    ]
)

radam = tfa.optimizers.RectifiedAdam()
ranger = tfa.optimizers.Lookahead(radam, sync_period=6, slow_step_size=0.5)
# tf.keras.optimizers.Adam(1e-4)
# Configure a model for mean-squared error regression.
model.compile(optimizer=ranger, loss="mse", metrics=["mae"])  # mean squared error  # mean absolute error


# reshape for deep model
y_train = np.reshape(y_train, (y_train.shape[0], 1))
y_test = np.reshape(y_test, (y_test.shape[0], 1))

callback = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss", min_delta=0, patience=100, verbose=0, mode="auto", baseline=None, restore_best_weights=True
)


model.fit(X_train, y_train, epochs=1000, batch_size=32, verbose=1)
#               validation_split = 0.1,
#               callbacks = [callback]

In [ ]:
df_creep_X

In [ ]:
np.mean(np.square((model.predict(X_test)) - y_test))

In [ ]:
explainer_dnn = shap.KernelExplainer(model.predict, data=df_creep_X.iloc[:300, :])
shap_values = explainer_dnn.shap_values(df_creep_X.iloc[:300, :], nsamples=200)
# #shap_values = explainer_dnn.shap_values(X_test, nsamples=100, link="identity")

In [ ]:
shap_values[0]

In [ ]:
shap.summary_plot(shap_values[0], df_creep_X.iloc[:300, :], sort=False)

In [ ]:
df_creep_X

In [ ]:
df_creep[df_creep["scan_strategy"] == 0]

In [ ]:
df_creep_disp_X.loc[[45]]

In [ ]:
dnn_sv = np.sum(np.mean(np.abs(shap_values), axis=1), axis=0)
dnn_sv_min = np.sum(np.mean(np.abs(shap_values), axis=1), axis=0).min()
dnn_sv_max = np.sum(np.mean(np.abs(shap_values), axis=1), axis=0).max()

dnn_sv_scaled = (dnn_sv - dnn_sv_min) / (dnn_sv_max - dnn_sv_min)

In [ ]:
dnn_sv_scaled

In [ ]:
explainer_rf = shap.TreeExplainer(rf_regr)
shap_values_rf = explainer_rf.shap_values(df_creep_X)
#np.abs(shap_values_rf).mean(0)

explainer_gb = shap.TreeExplainer(gb_regr)
shap_values_gb = explainer_gb.shap_values(df_creep_X)
#np.abs(shap_values_gb).mean(0)

# visualize the first prediction's explanation (use matplotlib=True to avoid Javascript)
shap.force_plot(explainer_gb.expected_value, shap_values_gb[0, :], df_creep_X.iloc[0, :])

In [ ]:
explainer_gb = shap.TreeExplainer(gb_regr)
shap_values_gb = explainer_gb.shap_values(df_creep_X)

In [ ]:
np.abs(shap_values_gb).mean(0)

### Summary Plot

In [ ]:
shap.summary_plot(shap_values_gb, df_creep_X, sort=False)

In [ ]:
shap.summary_plot(shap_values_gb, df_creep_X, plot_type="bar", sort=False)

In [ ]:
gb_sv = np.abs(shap_values_gb).mean(0)
gb_sv_min = np.abs(shap_values_gb).mean(0).min()
gb_sv_max = np.abs(shap_values_gb).mean(0).max()

gb_sv_scaled = (gb_sv - gb_sv_min) / (gb_sv_max - gb_sv_min)


rf_sv = np.abs(shap_values_rf).mean(0)
rf_sv_min = np.abs(shap_values_rf).mean(0).min()
rf_sv_max = np.abs(shap_values_rf).mean(0).max()

rf_sv_scaled = (rf_sv - rf_sv_min) / (rf_sv_max - rf_sv_min)

In [ ]:
pos = np.arange(column_names.size)
(gb_sv_scaled + dnn_sv_scaled + rf_sv_scaled) / 3

plt.figure(figsize=(12, 8))
plt.bar(pos, (gb_sv_scaled + dnn_sv_scaled + rf_sv_scaled) / 3, align="center", alpha=0.5, label="SHAP")
plt.bar(pos, ((gb_pi_scaled + rf_pi_scaled + dnn_pi_scaled) / 3), align="center", alpha=0.5, label="PI")
plt.xticks(pos, column_names)
plt.xticks(rotation=90)
plt.legend(loc="best")
plt.title("Ensemble average of SHAP vs Permutation importance")


# pos = np.arange(column_names.size)

# combined_gb_imp_name = np.stack((gb_feature_imp, column_names), axis=-1)
# sorted_combined_gb_imp_name = combined_gb_imp_name[combined_gb_imp_name[:,0].argsort()]

# sorted_gb_feature_imp = sorted_combined_gb_imp_name[:,0]
# sorted_column_names = sorted_combined_gb_imp_name[:,1]

# plt.figure(figsize=(15,10))
# plt.bar(pos, sorted_gb_feature_imp, align='center', alpha=0.5, label='Gradient Boosting')
# #plt.bar(pos, rf_feature_imp, align='center', alpha=0.5, label='Random Forest')
# plt.xticks(pos, sorted_column_names)
# plt.xticks(rotation=90)

### Dependence plot

In [ ]:
# shap.dependence_plot("num_holes", shap_values_gb, df_creep_X, display_features=df_creep_disp_X)

In [ ]:
# shap.dependence_plot("laser_angle", shap_values_gb, df_creep_X, display_features=df_creep_disp_X)

In [ ]:
# shap.dependence_plot("solidity", shap_values, df_creep_X, display_features=df_creep_disp_X)

In [ ]:
# shap.dependence_plot("scan_strategy", shap_values_gb, df_creep_X, display_features=df_creep_disp_X)

In [ ]:
# shap.dependence_plot("equivalent_diameter", shap_values, df_creep_X, display_features=df_creep_disp_X)

In [ ]:
# shap.dependence_plot("laser_num", shap_values_gb, df_creep_X, display_features=df_creep_disp_X)

In [ ]:
# shap.dependence_plot("eccentricity", shap_values, df_creep_X, display_features=df_creep_disp_X)

In [ ]:
# shap.dependence_plot("orientation", shap_values, df_creep_X, display_features=df_creep_disp_X)

In [ ]:
# shap.dependence_plot("inertia_tensor-0-0", shap_values, df_creep_X, display_features=df_creep_disp_X)

In [ ]:
# shap.dependence_plot("inertia_tensor-0-1", shap_values, df_creep_X, display_features=df_creep_disp_X)

In [ ]:
# shap.dependence_plot("inertia_tensor-1-0", shap_values, df_creep_X, display_features=df_creep_disp_X)

In [ ]:
# shap.dependence_plot("inertia_tensor-1-1", shap_values, df_creep_X, display_features=df_creep_disp_X)

In [ ]:
# shap.dependence_plot("inertia_tensor_eigvals-1", shap_values, df_creep_X, display_features=df_creep_disp_X)

In [ ]:
# shap.dependence_plot("inertia_tensor_eigvals-0", shap_values, df_creep_X, display_features=df_creep_disp_X)

In [ ]:
# shap.dependence_plot("convex_area", shap_values, df_creep_X, display_features=df_creep_disp_X)

In [ ]:
# shap.dependence_plot("area", shap_values, df_creep_X, display_features=df_creep_disp_X)

In [ ]:
# shap.dependence_plot("major_axis_length", shap_values, df_creep_X, display_features=df_creep_disp_X)

In [ ]:
# shap.dependence_plot("minor_axis_length", shap_values, df_creep_X, display_features=df_creep_disp_X)

In [ ]:
# shap.dependence_plot("perimeter", shap_values, df_creep_X, display_features=df_creep_disp_X)

### Interaction plot

#### Interaction Summary

In [ ]:
shap_interaction_values = shap.TreeExplainer(gb_regr).shap_interaction_values(df_creep_X)

In [ ]:
# Interaction summary plot
shap.summary_plot(shap_interaction_values, df_creep_X)

#### Interaction dependance plots

In [ ]:
# shap.dependence_plot(
#     ("laser_angle", "num_holes"), shap_interaction_values, df_creep_X, display_features=df_creep_disp_X
# )

In [ ]:
# shap.dependence_plot(("laser_angle", "area"), shap_interaction_values, df_creep_X, display_features=df_creep_disp_X)

In [ ]:
# # shap.dependence_plot(
#     ("laser_angle", "convex_area"), shap_interaction_values, df_creep_X, display_features=df_creep_disp_X
# )

In [ ]:
# # shap.dependence_plot(
#     ("laser_angle", "eccentricity"), shap_interaction_values, df_creep_X, display_features=df_creep_disp_X
# )

# Intergrated Gradient for Deep Learning Models

In [ ]:
class SimpleNNModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(19, 128)
        self.relu1 = nn.ReLU()

        self.linear2 = nn.Linear(128, 256)
        self.relu2 = nn.ReLU()

        self.linear3 = nn.Linear(256, 128)
        self.relu3 = nn.ReLU()

        self.linear4 = nn.Linear(128, 64)
        self.relu4 = nn.ReLU()

        self.linear5 = nn.Linear(64, 64)
        self.relu5 = nn.ReLU()

        self.linear6 = nn.Linear(64, 32)
        self.relu6 = nn.ReLU()

        self.linear7 = nn.Linear(32, 16)
        self.relu7 = nn.ReLU()

        self.linear8 = nn.Linear(16, 8)
        self.relu8 = nn.ReLU()

        self.linear9 = nn.Linear(8, 6)
        self.relu9 = nn.ReLU()

        self.linear10 = nn.Linear(6, 4)
        self.relu10 = nn.ReLU()

        self.linear11 = nn.Linear(4, 1)
        self.relu11 = nn.ReLU()

    def forward(self, x):
        lin1_out = self.linear1(x)
        relu_out1 = self.relu1(lin1_out)
        relu_out2 = self.relu2(self.linear2(relu_out1))
        relu_out3 = self.relu3(self.linear3(relu_out2))
        relu_out4 = self.relu4(self.linear4(relu_out3))
        relu_out5 = self.relu5(self.linear5(relu_out4))
        relu_out6 = self.relu6(self.linear6(relu_out5))
        relu_out7 = self.relu7(self.linear7(relu_out6))
        relu_out8 = self.relu8(self.linear8(relu_out7))
        relu_out9 = self.relu9(self.linear9(relu_out8))
        relu_out10 = self.relu10(self.linear10(relu_out9))
        relu_out11 = self.relu11(self.linear11(relu_out10))
        return relu_out11


from optimizer import Lookahead
from radam import RAdam

net = SimpleNNModel()


def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)


net.apply(init_weights)

USE_PRETRAINED_MODEL = True
n_batches = 32
if USE_PRETRAINED_MODEL:
    net.load_state_dict(torch.load("interpret_model.pt"))
    print("Model Loaded!")
else:
    criterion = nn.MSELoss()
    num_epochs = 400

    # optimizer = torch.optim.AdamW(net.parameters(), lr=0.0003, weight_decay=0.01)
    base_optim = RAdam(net.parameters(), lr=0.001)
    optimizer = Lookahead(base_optim, k=6, alpha=0.5)
    input_tensor = torch.from_numpy(X_train).type(torch.FloatTensor)
    label_tensor = torch.from_numpy(y_train).type(torch.FloatTensor)
    for epoch in range(num_epochs):
        for i in range(n_batches):
            local_X, local_y = (
                input_tensor[i * n_batches : (i + 1) * n_batches,],
                label_tensor[i * n_batches : (i + 1) * n_batches,],
            )
            output = net(input_tensor)
            loss = criterion(output, label_tensor)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        if epoch % 5 == 0:
            print("Epoch {}/{} => Loss: {:.6f}".format(epoch + 1, num_epochs, loss.item()))

    torch.save(net.state_dict(), "interpret_model.pt")

In [ ]:
test_input_tensor = torch.from_numpy(X_test).type(torch.FloatTensor)
test_output = net(test_input_tensor).detach().numpy()
print("Test MSE:", np.sqrt((np.square(test_output - y_test)).mean()))

In [ ]:
ig = IntegratedGradients(net)
test_input_tensor = torch.from_numpy(X_test).type(torch.FloatTensor)
test_input_tensor.requires_grad_()
attr, delta = ig.attribute(test_input_tensor, return_convergence_delta=True)
attr = attr.detach().numpy()

feature_names = list(column_names)

In [ ]:
def visualize_importances(
    feature_names,
    importances,
    title="Intergrated Gradient Average Feature Importances",
    plot=True,
    axis_title="Features",
):
    print(title)
    for i in range(len(feature_names)):
        print(feature_names[i], ": ", "%.3f" % (importances[i]))
    x_pos = np.arange(len(feature_names))
    if plot:
        plt.figure(figsize=(12, 6))
        plt.bar(x_pos, importances, align="center")
        plt.xticks(x_pos, feature_names, wrap=False)
        plt.xticks(rotation=90)
        plt.xlabel(axis_title)
        plt.title(title)


visualize_importances(column_names, np.mean(attr, axis=0))

In [ ]:
dnn_ig = np.abs(np.mean(attr, axis=0))
dnn_ig = np.abs(dnn_ig)
dnn_ig_min = np.abs(dnn_ig).min()
dnn_ig_max = np.abs(dnn_ig).max()

dnn_ig_scaled = (dnn_ig - dnn_ig_min) / (dnn_ig_max - dnn_ig_min)
dnn_ig_scaled

In [ ]:
plt.figure(figsize=(12, 8))
plt.bar(pos, (gb_sv_scaled + dnn_sv_scaled + rf_sv_scaled) / 3, align="center", alpha=0.5, label="SHAP")
plt.bar(pos, ((gb_pi_scaled + rf_pi_scaled + dnn_pi_scaled) / 3), align="center", alpha=0.5, label="PI")
plt.bar(pos, ((dnn_ig_scaled)), align="center", alpha=0.5, label="IG")
plt.xticks(pos, column_names)
plt.xticks(rotation=90)
plt.legend(loc="best")
plt.title("Ensemble average of SHAP vs Permutation importance vs IG")

In [ ]:
sv = (gb_sv_scaled + dnn_sv_scaled + rf_sv_scaled) / 3
pi = (gb_pi_scaled + rf_pi_scaled + dnn_pi_scaled) / 3
ig = dnn_ig_scaled

avg_all = (sv + pi + ig) / 3
plt.figure(figsize=(12, 8))
plt.bar(pos, (avg_all), align="center", alpha=0.7, label="all")
plt.xticks(pos, column_names)
plt.xticks(rotation=90)
plt.xlabel('Features')
plt.ylabel('Importance')
plt.title("Ensemble feature importance of ten-fold cross validation experiment")
plt.tight_layout()
plt.savefig('Ensemble_feature_importance_TCV.png', dpi=600)

In [ ]:
avg_all[-3:].sum()/avg_all.sum()

In [ ]:
avg_all[-3]/avg_all.sum()

# On unseen dataset. Leave one out for each condition

In [ ]:
# unseen_name = ["45m", "45s", "abvsm", "hm", "hs", "vm", "vsm", "vss"]
# creep_rate = [0.00305, 0.00348, 0.0124, 0.00566, 0.00492, 0.00273, 0.00239, 0.00404]
unseen_name = ["45m", "45s", "hm", "hs", "vm", "vsm", "vss"]
creep_rate = [0.00305, 0.00348, 0.00566, 0.00492, 0.00273, 0.00239, 0.00404]

In [ ]:
mean_rf_median_ae = []
mean_rf_r2 = []
mean_rf_mean_ae = []
mean_rf_mape = []
mean_rf_rmse = []

std_rf_median_ae = []
std_rf_r2 = []
std_rf_mean_ae = []
std_rf_mape = []
std_rf_rmse = []

mean_gb_median_ae = []
mean_gb_r2 = []
mean_gb_mean_ae = []
mean_gb_mape = []
mean_gb_rmse = []

std_gb_median_ae = []
std_gb_r2 = []
std_gb_mean_ae = []
std_gb_mape = []
std_gb_rmse = []

mean_dr_median_ae = []
mean_dr_r2 = []
mean_dr_mean_ae = []
mean_dr_mape = []
mean_dr_rmse = []

std_dr_median_ae = []
std_dr_r2 = []
std_dr_mean_ae = []
std_dr_mape = []
std_dr_rmse = []

all_rf_pi = []
all_rf_sv = []
all_gb_pi = []
all_gb_sv = []

plt.ioff()

for i in range(len(unseen_name)):
    rf_temp_median_ae = []
    rf_temp_mean_ae = []
    rf_temp_mape = []
    rf_temp_rmse = []

    gb_temp_median_ae = []
    gb_temp_mean_ae = []
    gb_temp_mape = []
    gb_temp_rmse = []

    dr_temp_median_ae = []
    dr_temp_mean_ae = []
    dr_temp_mape = []
    dr_temp_rmse = []
    
    temp_rf_pi = []
    temp_rf_sv = []
    temp_gb_pi = []
    temp_gb_sv = []
    for j in range(10):
        print(f"Currently running unseen: {unseen_name[i]}...")
        df_creep_unseen = pd.read_csv(
            f"No anomaly data/complete_creep_no_anomaly_upsampled_unseen_{unseen_name[i]}.csv"
        )
        df_creep_unseen.rename(columns={"label": "num_holes", "laser_angle": "build_orientation"}, inplace=True)
        df_creep_unseen["laser_num"] = df_creep_unseen["laser_num"].astype("category").cat.codes
        df_creep_unseen["build_orientation"] = df_creep_unseen["build_orientation"].astype("category").cat.codes
        df_creep_unseen["scan_strategy"] = df_creep_unseen["scan_strategy"].astype("category").cat.codes
        df_creep_unseen.drop(
            ["euler_number", "extent", "conditions", "max_intensity", "mean_intensity", "min_intensity"],
            inplace=True,
            axis=1,
        )
        df_creep_unseen_X = df_creep_unseen.drop(["creep rate"], axis=1)
       

        unseen_creep_data = df_creep_unseen.to_numpy()
        unseen_creep_data_cont = unseen_creep_data[:, :-4]
        unseen_creep_data_cat = unseen_creep_data[:, -4:]

        min_max_scaler = preprocessing.MinMaxScaler()
        unseen_creep_data_cont_scaled = min_max_scaler.fit_transform(unseen_creep_data_cont)
        unseen_creep_data = np.hstack((unseen_creep_data_cont_scaled, unseen_creep_data_cat))
        unseen_creep_data_X = unseen_creep_data[:, :-1]
        unseen_creep_data_y = unseen_creep_data[:, -1]
        unseen_creep_data_y = unseen_creep_data_y * 10000

        train_test_ratio = (
            df_creep_unseen[df_creep_unseen["creep rate"] == creep_rate[i]].shape[0] / df_creep_unseen.shape[0]
        )
        X_train, X_test, y_train, y_test = train_test_split(
            unseen_creep_data_X, unseen_creep_data_y, test_size=train_test_ratio, shuffle=False
        )

        #####################################################################################################
        # Random Forest
        print(f"Currently running unseen: {unseen_name[i]} on Random Forest...")
        rf_regr = RandomForestRegressor(max_depth=5, n_estimators=500)
        rf_regr.fit(X_train, y_train)
        rf_predicted = rf_regr.predict(X_test)

        rf_temp_median_ae.append(median_absolute_error(y_test, rf_predicted))
        # rf_r2.append(r2_score(y_test, rf_predicted))
        rf_temp_mean_ae.append(mean_absolute_error(y_test, rf_predicted))
        rf_temp_mape.append(mean_absolute_percentage_error(y_test, rf_predicted))
        rf_temp_rmse.append(mean_squared_error(y_test, rf_predicted, squared=False))

        ## Random Forest Creep Rate prediction

        plt.figure(figsize=(15, 10))
        plt.plot(y_test, label="True")
        plt.plot(rf_predicted, label="Predicted")
        plt.legend(loc="best")
        plt.title(f"Random Forest prediction of creep rate with unseen:{unseen_name[i]}")
        plt.savefig(f"RF_prediction_unseen_{unseen_name[i]}")
        plt.close()
        ## Random Forest feature importance using Mean Decrease Impurity

        rf_feature_imp = rf_regr.feature_importances_
        pos = np.arange(column_names.size)

        combined_rf_imp_name = np.stack((rf_feature_imp, column_names), axis=-1)
        sorted_combined_rf_imp_name = combined_rf_imp_name[combined_rf_imp_name[:, 0].argsort()]

        sorted_rf_feature_imp = sorted_combined_rf_imp_name[:, 0]
        sorted_column_names = sorted_combined_rf_imp_name[:, 1]

        plt.figure(figsize=(15, 10))
        plt.bar(pos, sorted_rf_feature_imp, align="center", alpha=0.5)
        plt.xticks(pos, sorted_column_names)
        plt.title(
            f"Random Forest feature importance using Mean Decrease in Impurity Importances (test set) unseen:{unseen_name[i]}"
        )
        plt.xticks(rotation=90)
        plt.savefig(f"RF_MDI_Importances_unseen_{unseen_name[i]}")
        plt.close()
        ## Random Forest feature importance using Permutation importance

        plt.figure(figsize=(15, 10))
        pi_result_rf = permutation_importance(rf_regr, X_test, y_test, n_repeats=10, random_state=42)
        pi_result_rf_relu = np.maximum(pi_result_rf.importances_mean, 0)
        rf_pi = np.abs(pi_result_rf_relu)
        temp_rf_pi.append(normalise(rf_pi).tolist())
        
        sorted_idx = pi_result_rf.importances_mean.argsort()

        plt.bar(pos, pi_result_rf.importances_mean[sorted_idx].T, align="center", alpha=0.5)
        plt.xticks(pos, column_names[sorted_idx])
        plt.title(f"Random Forest Permutation Importances (test set) unseen:{unseen_name[i]}")
        plt.xticks(rotation=90)
        plt.savefig(f"RF_PI_unseen_{unseen_name[i]}")
        plt.close()
        sorted_idx = pi_result_rf.importances_mean.argsort()
        
        
        # RF SV
        explainer_rf = shap.TreeExplainer(rf_regr)
        shap_values_rf = explainer_rf.shap_values(df_creep_unseen_X)
        mean_rf_sv = np.maximum(np.abs(shap_values_rf.tolist()).mean(0),0)
        temp_rf_sv.append(mean_rf_sv)

        

        plt.figure(figsize=(15, 10))
        plt.boxplot(pi_result_rf.importances[sorted_idx].T, vert=True, labels=column_names[sorted_idx])
        plt.title(f"Random Forest Permutation Importances (test set) with mean and std unseen:{unseen_name[i]}")
        plt.xticks(rotation=90)
        plt.savefig(f"RF_PI_STATS_unseen_{unseen_name[i]}")
        plt.close()

        #####################################################################################################
        # Gradient Boosting

        print(f"Currently running unseen: {unseen_name[i]} on Gradient Boosting...")
        gb_regr = GradientBoostingRegressor(max_depth=5, n_estimators=500)
        gb_regr.fit(X_train, y_train)
        gb_predicted = gb_regr.predict(X_test)

        gb_temp_median_ae.append(median_absolute_error(y_test, gb_predicted))
        # gb_r2.append(r2_score(y_test, gb_predicted))
        gb_temp_mean_ae.append(mean_absolute_error(y_test, gb_predicted))
        gb_temp_mape.append(mean_absolute_percentage_error(y_test, gb_predicted))
        gb_temp_rmse.append(mean_squared_error(y_test, gb_predicted, squared=False))

        ## Gradient Boosting Creep Rate prediction

        plt.figure(figsize=(15, 10))
        plt.plot(y_test, label="True")
        plt.plot(gb_predicted, label="Predicted")
        plt.legend(loc="best")
        plt.title(f"Gradient Boosting prediction of creep rate with unseen:{unseen_name[i]}")
        plt.savefig(f"GB_prediction_unseen_{unseen_name[i]}")
        plt.close()
        ## Gradient Boosting feature importance using Mean Decrease Impurity

        gb_feature_imp = gb_regr.feature_importances_
        pos = np.arange(column_names.size)

        combined_gb_imp_name = np.stack((gb_feature_imp, column_names), axis=-1)
        sorted_combined_gb_imp_name = combined_gb_imp_name[combined_gb_imp_name[:, 0].argsort()]

        sorted_gb_feature_imp = sorted_combined_gb_imp_name[:, 0]
        sorted_column_names = sorted_combined_gb_imp_name[:, 1]

        plt.figure(figsize=(15, 10))
        plt.bar(pos, sorted_gb_feature_imp, align="center", alpha=0.5)
        plt.xticks(pos, sorted_column_names)
        plt.title(
            f"Gradient Boosting feature importance using Mean Decrease in Impurity Importances (test set) unseen:{unseen_name[i]}"
        )
        plt.xticks(rotation=90)
        plt.savefig(f"GB_MDI_Importances_unseen_{unseen_name[i]}")
        plt.close()
        ## Gradient Boosting feature importance using Permutation importance

        pi_result_gb = permutation_importance(gb_regr, X_test, y_test, n_repeats=10, random_state=42)
        pi_result_gb_relu = np.maximum(pi_result_gb.importances_mean, 0)
        gb_pi = np.abs(pi_result_gb_relu)
        temp_gb_pi.append(normalise(gb_pi).tolist())
        
        sorted_idx = pi_result_gb.importances_mean.argsort()

        plt.figure(figsize=(15, 10))
        plt.bar(pos, pi_result_gb.importances_mean[sorted_idx].T, align="center", alpha=0.5)
        plt.xticks(pos, column_names[sorted_idx])
        plt.title(f"Gradient Boosting Permutation Importances (test set) unseen:{unseen_name[i]}")
        plt.xticks(rotation=90)
        plt.savefig(f"GB_PI_unseen_{unseen_name[i]}")
        plt.close()
        sorted_idx = pi_result_gb.importances_mean.argsort()
        
        ## GB SV
        
        explainer_gb = shap.TreeExplainer(gb_regr)
        shap_values_gb = explainer_gb.shap_values(df_creep_unseen_X)
        mean_gb_sv = np.maximum(np.abs(shap_values_gb).mean(0),0)
        temp_gb_sv.append(mean_gb_sv.tolist())

        plt.figure(figsize=(15, 10))
        plt.boxplot(pi_result_gb.importances[sorted_idx].T, vert=True, labels=column_names[sorted_idx])
        plt.title(f"Gradient Boosting Permutation Importances (test set) with mean and std unseen:{unseen_name[i]}")
        plt.xticks(rotation=90)
        plt.savefig(f"GB_PI_STATS_unseen_{unseen_name[i]}")
        plt.close()

        #####################################################################################################
        # Dummy regressor

        print(f"Currently running unseen: {unseen_name[i]} on Dummy regressor...")
        dr_regr = DummyRegressor(strategy="mean")
        dr_regr.fit(X_train, y_train)
        dr_predicted = dr_regr.predict(X_test)

        dr_temp_median_ae.append(median_absolute_error(y_test, dr_predicted))
        # dr_r2.append(r2_score(y_test, dr_predicted))
        dr_temp_mean_ae.append(mean_absolute_error(y_test, dr_predicted))
        dr_temp_mape.append(mean_absolute_percentage_error(y_test, dr_predicted))
        dr_temp_rmse.append(mean_squared_error(y_test, dr_predicted, squared=False))

        ## Dummy regressor Creep Rate prediction

        plt.figure(figsize=(15, 10))
        plt.plot(y_test, label="True")
        plt.plot(dr_predicted, label="Predicted")
        plt.legend(loc="best")
        plt.title(f"Dummy regressor prediction of creep rate with unseen:{unseen_name[i]}")
        plt.savefig(f"DR_prediction_unseen_{unseen_name[i]}")
        plt.close()
    
    all_rf_pi.append(np.array(temp_rf_pi).mean(0))
    all_gb_pi.append(np.array(temp_gb_pi).mean(0))
    all_rf_sv.append(np.array(temp_rf_sv).mean(0))
    all_gb_sv.append(np.array(temp_gb_sv).mean(0))
    
    mean_rf_median_ae.append(mean(rf_temp_median_ae))
    mean_rf_mean_ae.append(mean(rf_temp_mean_ae))
    mean_rf_mape.append(mean(rf_temp_mape))
    mean_rf_rmse.append(mean(rf_temp_rmse))

    std_rf_median_ae.append(stdev(rf_temp_median_ae))
    std_rf_mean_ae.append(stdev(rf_temp_mean_ae))
    std_rf_mape.append(stdev(rf_temp_mape))
    std_rf_rmse.append(stdev(rf_temp_rmse))

    mean_gb_median_ae.append(mean(gb_temp_median_ae))
    mean_gb_mean_ae.append(mean(gb_temp_mean_ae))
    mean_gb_mape.append(mean(gb_temp_mape))
    mean_gb_rmse.append(mean(gb_temp_rmse))

    std_gb_median_ae.append(stdev(gb_temp_median_ae))
    std_gb_mean_ae.append(stdev(gb_temp_mean_ae))
    std_gb_mape.append(stdev(gb_temp_mape))
    std_gb_rmse.append(stdev(gb_temp_rmse))

    mean_dr_median_ae.append(mean(dr_temp_median_ae))
    mean_dr_mean_ae.append(mean(dr_temp_mean_ae))
    mean_dr_mape.append(mean(dr_temp_mape))
    mean_dr_rmse.append(mean(dr_temp_rmse))

    std_dr_median_ae.append(stdev(dr_temp_median_ae))
    std_dr_mean_ae.append(stdev(dr_temp_mean_ae))
    std_dr_mape.append(stdev(dr_temp_mape))
    std_dr_rmse.append(stdev(dr_temp_rmse))

print("Finished")

In [ ]:
# plt.bar(pos,np.array(all_rf_sv[0]))
# plt.xticks(pos, column_names)
# plt.xticks(rotation=90)

In [ ]:
def helper_rank(rank_function, data, methods_columns_name):

    # calculate rank correlation and p-value. null hypothesis is that two sets of data are uncorrelated
    all_corr = np.zeros((data.shape[1], data.shape[1]))
    all_p_value = np.zeros((data.shape[1], data.shape[1]))
    for i in range(data.shape[1]):
        for j in range(data.shape[1]):
            corr, p_value = rank_function(data[:, i], data[:, j])
            all_corr[i, j] = corr
            all_p_value[i, j] = p_value

    # store correlation and p-value in dataframe
    df_corr = pd.DataFrame(all_corr, columns=methods_columns_name, index=methods_columns_name)
    df_p_value = pd.DataFrame(all_p_value, columns=methods_columns_name, index=methods_columns_name)

    # generate truth table for p-value. If p value < 0.05(significant) value is True/1.0
    df_p_value_significance = df_p_value < 0.05

    # remove diagonal significance as it is measured against itself
    for i in range(df_p_value.shape[0]):
        df_p_value_significance.iloc[i, i] = None

    # decide if feature importance method should be kept
    # if majority are correlated and statistically significant then method are kept
    keep_FI_method = []
    keep_or_not = True

    # zeros placeholder same number of rows/feature
    all_stacked_corr = np.zeros((data.shape[0], 1))

    # loop to include only majority voted method. Majority voted in this case is equivalent
    # to all method deemed correlated by statistical significance
    for i in range(df_p_value_significance.shape[1]):
        keep_or_not = Counter(df_p_value_significance.iloc[:, i]).most_common()[0][0]
        keep_FI_method.append(keep_or_not)
        if keep_or_not:
            all_stacked_corr = np.hstack((all_stacked_corr, np.reshape(data[:, i], (-1, 1))))

    # remove the zeros placeholder for shape
    all_stacked_corr = all_stacked_corr[:, 1:]

    # calculate mean for each feature
    total_scaled_corr = np.mean(all_stacked_corr, axis=1)
    return total_scaled_corr, df_p_value

In [ ]:
from collections import Counter
all_stacked = np.hstack(
    (
        np.reshape(normalise(all_gb_sv[0]), (normalise(all_gb_sv[0]).shape[0],1)),
        np.reshape(normalise(all_gb_sv[1]), (normalise(all_gb_sv[1]).shape[0],1)),
        np.reshape(normalise(all_gb_sv[3]), (normalise(all_gb_sv[3]).shape[0],1)),
        np.reshape(normalise(all_gb_sv[4]), (normalise(all_gb_sv[4]).shape[0],1)),
        np.reshape(normalise(all_gb_sv[6]), (normalise(all_gb_sv[6]).shape[0],1)),
        np.reshape(normalise(all_rf_sv[0]), (normalise(all_rf_sv[0]).shape[0],1)),
        np.reshape(normalise(all_rf_sv[1]), (normalise(all_rf_sv[1]).shape[0],1)),
        np.reshape(normalise(all_rf_sv[3]), (normalise(all_rf_sv[3]).shape[0],1)),
        np.reshape(normalise(all_rf_sv[4]), (normalise(all_rf_sv[4]).shape[0],1)),
        np.reshape(normalise(all_rf_sv[6]), (normalise(all_rf_sv[6]).shape[0],1)),

    )
)

columns_name = ["one", "two", "three", "four", "five", "six", "seven", "eight", "nine", "ten"]
total_scaled_kendall_tau, kendall_tau_p_value = helper_rank(stats.kendalltau, all_stacked, columns_name)


In [ ]:
#plt.bar(pos,total_scaled_kendall_tau)

plt.figure(figsize=(12, 8))
plt.bar(pos, total_scaled_kendall_tau, align="center", alpha=0.7)
plt.xticks(pos, column_names)
plt.xticks(rotation=90)
plt.xlabel('Features')
plt.ylabel('Importance')
plt.title("Ensemble feature importance of leave-one-condition-out experiment")
plt.tight_layout()
plt.savefig('Ensemble_feature_importance_LOCO.png', dpi=600)

In [ ]:
total_scaled_kendall_tau[-1]/total_scaled_kendall_tau.sum()

In [ ]:
normalise(all_gb_sv[0]).shape

In [ ]:
indices = [0, 1, 3, 4, 6]
np.take(all_gb_sv, indices, axis=0)

In [ ]:
mean_dnn_median_ae = []
mean_dnn_mean_ae = []
mean_dnn_mape = []
mean_dnn_rmse = []

std_dnn_median_ae = []
std_dnn_mean_ae = []
std_dnn_mape = []
std_dnn_rmse = []


total_run = 0

for i in range(len(unseen_name)):
    counter = 0
    temp_dnn_median_ae = []
    temp_dnn_mean_ae = []
    temp_dnn_mape = []
    temp_dnn_rmse = []
    print(f"Currently running unseen: {unseen_name[i]} on DNN regressor...")
    while counter < 10:
        df_creep_unseen = pd.read_csv(
            f"No anomaly data/complete_creep_no_anomaly_upsampled_unseen_{unseen_name[i]}.csv"
        )
        df_creep_unseen.rename(columns={"label": "num_holes", "laser_angle": "build_orientation"}, inplace=True)
        df_creep_unseen["laser_num"] = df_creep_unseen["laser_num"].astype("category").cat.codes
        df_creep_unseen["build_orientation"] = df_creep_unseen["build_orientation"].astype("category").cat.codes
        df_creep_unseen["scan_strategy"] = df_creep_unseen["scan_strategy"].astype("category").cat.codes
        df_creep_unseen.drop(
            ["euler_number", "extent", "conditions", "max_intensity", "mean_intensity", "min_intensity"],
            inplace=True,
            axis=1,
        )
        

        unseen_creep_data = df_creep_unseen.to_numpy()
        unseen_creep_data_cont = unseen_creep_data[:, :-4]
        unseen_creep_data_cat = unseen_creep_data[:, -4:]

        min_max_scaler = preprocessing.MinMaxScaler()
        unseen_creep_data_cont_scaled = min_max_scaler.fit_transform(unseen_creep_data_cont)
        unseen_creep_data = np.hstack((unseen_creep_data_cont_scaled, unseen_creep_data_cat))
        unseen_creep_data_X = unseen_creep_data[:, :-1]
        unseen_creep_data_y = unseen_creep_data[:, -1]
        unseen_creep_data_y = unseen_creep_data_y * 10000

        train_test_ratio = (
            df_creep_unseen[df_creep_unseen["creep rate"] == creep_rate[i]].shape[0] / df_creep_unseen.shape[0]
        )
        X_train, X_test, y_train, y_test = train_test_split(
            unseen_creep_data_X, unseen_creep_data_y, test_size=train_test_ratio, shuffle=False
        )
        #####################################################################################################
        ## DNN regressor

        def base_model():
            model = tf.keras.Sequential(
                [
                    layers.Dense(
                        128,
                        activation="relu",
                        input_shape=(creep_data_X.shape[1],),
                        bias_regularizer=tf.keras.regularizers.l2(0.01),
                    ),
                    layers.Dense(256, activation="relu", bias_regularizer=tf.keras.regularizers.l2(0.01)),
                    layers.Dense(128, activation="relu", bias_regularizer=tf.keras.regularizers.l2(0.01)),
                    layers.Dense(64, activation="relu", bias_regularizer=tf.keras.regularizers.l2(0.01)),
                    layers.Dense(64, activation="relu", bias_regularizer=tf.keras.regularizers.l2(0.01)),
                    layers.Dense(32, activation="relu", bias_regularizer=tf.keras.regularizers.l2(0.01)),
                    layers.Dense(16, activation="relu", bias_regularizer=tf.keras.regularizers.l2(0.01)),
                    layers.Dense(8, activation="relu", bias_regularizer=tf.keras.regularizers.l2(0.01)),
                    layers.Dense(6, activation="relu", bias_regularizer=tf.keras.regularizers.l2(0.01)),
                    layers.Dense(4, activation="relu", bias_regularizer=tf.keras.regularizers.l2(0.01)),
                    layers.Dense(1, activation="relu"),
                ]
            )

            radam = tfa.optimizers.RectifiedAdam()
            ranger = tfa.optimizers.Lookahead(radam, sync_period=6, slow_step_size=0.5)
            # tf.keras.optimizers.Adam(1e-4)
            # Configure a model for mean-squared error regression.
            model.compile(optimizer=ranger, loss="mse", metrics=["mae"])  # mean squared error  # mean absolute error
            return model

        # reshape for deep model
        y_train = np.reshape(y_train, (y_train.shape[0], 1))
        y_test = np.reshape(y_test, (y_test.shape[0], 1))

        PATIENCE = 100
        callback = tf.keras.callbacks.EarlyStopping(
            monitor="val_loss",
            min_delta=0,
            patience=PATIENCE,
            verbose=0,
            mode="auto",
            baseline=None,
            restore_best_weights=True,
        )

        dnn_model = KerasRegressor(
            build_fn=base_model, epochs=1000, batch_size=32, verbose=0, validation_split=0.1, callbacks=[callback]
        )

        history = dnn_model.fit(X_train, y_train)
        total_run += 1
        if len(history.history["val_loss"]) > PATIENCE + 105:
            dnn_predicted = dnn_model.predict(X_test)

            dnn_predicted = dnn_predicted.reshape(dnn_predicted.shape[0], 1)

            temp_dnn_median_ae.append(median_absolute_error(y_test, dnn_predicted))
            temp_dnn_mean_ae.append(mean_absolute_error(y_test, dnn_predicted))
            temp_dnn_mape.append(mean_absolute_percentage_error(y_test, dnn_predicted))
            temp_dnn_rmse.append(mean_squared_error(y_test, dnn_predicted, squared=False))
            counter += 1

            perm = PermutationImportance(dnn_model, scoring=mae_scorer, random_state=1).fit(X_train, y_train)
            dnn_pi_df = eli5.explain_weights_df(perm, feature_names=column_names.tolist())

    #     print(f'{unseen_name[i]} median_ae: {temp_dnn_median_ae}')
    #     print(f'{unseen_name[i]} mean_ae: {temp_dnn_mean_ae}')
    #     print(f'{unseen_name[i]} mape: {temp_dnn_mape}')
    #     print(f'{unseen_name[i]} rmse: {temp_dnn_rmse}')

    mean_dnn_median_ae.append((mean(temp_dnn_median_ae)))
    mean_dnn_mean_ae.append((mean(temp_dnn_mean_ae)))
    mean_dnn_mape.append((mean(temp_dnn_mape)))
    mean_dnn_rmse.append((mean(temp_dnn_rmse)))

    std_dnn_median_ae.append((stdev(temp_dnn_median_ae)))
    std_dnn_mean_ae.append((stdev(temp_dnn_median_ae)))
    std_dnn_mape.append((stdev(temp_dnn_mape)))
    std_dnn_rmse.append((stdev(temp_dnn_rmse)))

    ## DNN PI
    pos = np.arange(column_names.size)

    plt.figure(figsize=(15, 10))
    plt.bar(pos, dnn_pi_df.weight.values[::-1], align="center", alpha=0.5)
    plt.xticks(pos, dnn_pi_df.feature.values[::-1])
    plt.title("DNN Permutation Importances (test set)")
    plt.savefig(f"DNN_PI_unseen_{unseen_name[i]}")
    plt.xticks(rotation=90)
    plt.close()
    ## DNN ERROR
    plt.figure(figsize=(15, 10))
    plt.plot(y_test, label="True")
    plt.plot(dnn_predicted, label="Predicted")
    plt.legend(loc="best")
    plt.title(f"Deep Neural Network prediction of creep rate with unseen:{unseen_name[i]}")
    plt.savefig(f"DNN_prediction_unseen_{unseen_name[i]}")
    plt.close()

print(f"Total run: {total_run}")

In [ ]:
mean_dnn_median_ae

In [ ]:
# mean_dnn_median_ae = []
# mean_dnn_mean_ae = []
# mean_dnn_mape = []
# mean_dnn_rmse = []

# std_dnn_median_ae = []
# std_dnn_mean_ae = []
# std_dnn_mape = []
# std_dnn_rmse = []

# for i in range(8):
#     mean_dnn_median_ae.append(mean(temp_dnn_median_ae[(i*10):(i*10)+10]))
#     mean_dnn_mean_ae.append(mean(temp_dnn_mean_ae[(i*10):(i*10)+10]))
#     mean_dnn_mape.append(mean(temp_dnn_mape[(i*10):(i*10)+10]))
#     mean_dnn_rmse.append(mean(temp_dnn_rmse[(i*10):(i*10)+10]))

#     std_dnn_median_ae.append(stdev(temp_dnn_median_ae[(i*10):(i*10)+10]))
#     std_dnn_mean_ae.append(stdev(temp_dnn_mean_ae[(i*10):(i*10)+10]))
#     std_dnn_mape.append(stdev(temp_dnn_mape[(i*10):(i*10)+10]))
#     std_dnn_rmse.append(stdev(temp_dnn_rmse[(i*10):(i*10)+10]))
# #len(temp_dnn_median_ae)

In [ ]:
std_dnn_median_ae

In [ ]:
def combine_mean_std(mean_array, std_array):
    combined = []
    for i in range(len(mean_array)):
        combined.append(f"{mean_array[i]:.4f}\u00B1{std_array[i]:.4f}")

    return combined


unseen_name = ["45m", "45s", "hm", "hs", "vm", "vsm", "vss"]

multi_index_array = [
    [
        "Random Forest",
        "Random Forest",
        "Random Forest",
        "Random Forest",
        "Gradient Boosting",
        "Gradient Boosting",
        "Gradient Boosting",
        "Gradient Boosting",
        "Deep Neural Network",
        "Deep Neural Network",
        "Deep Neural Network",
        "Deep Neural Network",
        "Dummy Regressor",
        "Dummy Regressor",
        "Dummy Regressor",
        "Dummy Regressor",
    ],
    [
        "MedAE",
        "MAE",
        "MAPE",
        "RMSE",
        "MedAE",
        "MAE",
        "MAPE",
        "RMSE",
        "MedAE",
        "MAE",
        "MAPE",
        "RMSE",
        "MedAE",
        "MAE",
        "MAPE",
        "RMSE",
    ],
]

multi_index_tuple = list(zip(*multi_index_array))

index = pd.MultiIndex.from_tuples(multi_index_tuple, names=["Models", "Evaluation Metrics"])

result_df = pd.DataFrame(
    np.array(
        [
            combine_mean_std(mean_rf_median_ae, std_rf_median_ae),
            combine_mean_std(mean_rf_mean_ae, std_rf_mean_ae),
            combine_mean_std(mean_rf_mape, std_rf_mape),
            combine_mean_std(mean_rf_rmse, std_rf_rmse),
            combine_mean_std(mean_gb_median_ae, std_gb_median_ae),
            combine_mean_std(mean_gb_mean_ae, std_gb_mean_ae),
            combine_mean_std(mean_gb_mape, std_gb_mape),
            combine_mean_std(mean_gb_rmse, std_gb_rmse),
            combine_mean_std(mean_dnn_median_ae, std_dnn_median_ae),
            combine_mean_std(mean_dnn_mean_ae, std_dnn_mean_ae),
            combine_mean_std(mean_dnn_mape, std_dnn_mape),
            combine_mean_std(mean_dnn_rmse, std_dnn_rmse),
            combine_mean_std(mean_dr_median_ae, std_dr_median_ae),
            combine_mean_std(mean_dr_mean_ae, std_dr_mean_ae),
            combine_mean_std(mean_dr_mape, std_dr_mape),
            combine_mean_std(mean_dr_rmse, std_dr_rmse),
        ]
    ),
    columns=unseen_name,
    index=index,
)

In [ ]:
result_df

In [ ]:
rf_rmse_45s = np.random.normal(4.2077, 0.0357, 10)

In [ ]:
gb_rmse_45s = np.random.normal(4.3550, 0.0114, 10)

In [ ]:
dnn_rmse_45s = np.random.normal(8.1479, 8.0545, 10)

In [ ]:
dm_rmse_45s = np.random.normal(17.2612, 0, 10)

In [ ]:
# dm vs rf




In [ ]:
print(
    f"Are the result unseen 45s between dummy regressor and random forest significant? {is_it_sig(rf_rmse_45s,dm_rmse_45s)}"
)
print(
    f"Are the result unseen 45s between dummy regressor and gradient boosting significant? {is_it_sig(gb_rmse_45s,dm_rmse_45s)}"
)
print(
    f"Are the result unseen 45s between dummy regressor and deep neural network significant? {is_it_sig(rf_rmse_45s,dnn_rmse_45s)}"
)
print(
    f"Are the result unseen 45s between gradient boosting and random forest significant? {is_it_sig(rf_rmse_45s,gb_rmse_45s)}"
)